In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
sales = pd.read_csv('../data/sales.csv', index_col=0)
online = pd.read_csv('../data/online.csv', index_col=0)
all_sales = pd.concat([sales, online], ignore_index=True)

In [3]:
catalog = pd.read_csv('../data/catalog.csv', index_col=0)
test = pd.read_csv('../data/test.csv', sep=';')
test.head()

,row_id,item_id,store_id,date
0,0,c578da8e8841,1,27.09.2024
1,1,c578da8e8841,1,28.09.2024
2,2,c578da8e8841,1,29.09.2024
3,3,c578da8e8841,1,30.09.2024
4,4,c578da8e8841,1,01.10.2024


In [4]:
# Group by date, store_id, item_id and aggregate
all_sales = all_sales.groupby(['date', 'store_id', 'item_id']).agg({
    'quantity': 'sum',
    'price_base': 'mean'
}).reset_index()
all_sales.head()


,date,store_id,item_id,quantity,price_base
0,2022-08-28,1,001829cb707d,7.0,134.76
1,2022-08-28,1,0022b986c8f0,2.0,59.90
2,2022-08-28,1,002f51c34a7a,4.0,83.11
3,2022-08-28,1,0052403cd09c,1.0,45.70
4,2022-08-28,1,005addd8096b,2.0,77.25


In [5]:
# Convert date to datetime
all_sales['date'] = pd.to_datetime(all_sales['date'])

# Create basic time features
all_sales['year'] = all_sales['date'].dt.year
all_sales['month'] = all_sales['date'].dt.month
all_sales['day'] = all_sales['date'].dt.day
all_sales['day_of_week'] = all_sales['date'].dt.dayofweek

In [6]:
# Fill missing values in catalog
catalog['dept_name'] = catalog['dept_name'].fillna('Unknown')
catalog['class_name'] = catalog['class_name'].fillna('Unknown')

# Merge with catalog and stores
all_sales = all_sales.merge(catalog[['item_id', 'dept_name', 'class_name']], 
                            on='item_id', how='left')
stores = pd.read_csv('../data/stores.csv', index_col=0)
all_sales = all_sales.merge(stores[['store_id', 'format', 'city']], 
                            on='store_id', how='left')

In [7]:
all_sales.head()

,date,store_id,item_id,quantity,price_base,year,month,day,day_of_week,dept_name,class_name,format,city
0,2022-08-28,1,001829cb707d,7.0,134.76,2022,8,28,6,СЛИВКИ,СЛИВКИ,Format-1,City1
1,2022-08-28,1,0022b986c8f0,2.0,59.90,2022,8,28,6,"СПЕЦИИ,ПРИПРАВА",МОНОСПЕЦИИ,Format-1,City1
2,2022-08-28,1,002f51c34a7a,4.0,83.11,2022,8,28,6,СОВРЕМЕННАЯ МОЛОЧНАЯ КАТЕГОРИЯ,ДЕСЕРТЫ,Format-1,City1
3,2022-08-28,1,0052403cd09c,1.0,45.70,2022,8,28,6,ДЕТСКИЕ МОЛОЧНЫЕ ПРОДУКТЫ,ДЕТСКИЕ ПРОДУКТЫ СТАРШЕ 3-Х ЛЕТ,Format-1,City1
4,2022-08-28,1,005addd8096b,2.0,77.25,2022,8,28,6,СЛАДКИЕ МУЧНЫЕ ИЗДЕЛИЯ,ПЕЧЕНЬЕ,Format-1,City1


In [ ]:
# Convert numeric columns to object type
all_sales['date'] = all_sales['date'].astype(str)
all_sales['store_id'] = all_sales['store_id'].astype(str)
all_sales['year'] = all_sales['year'].astype(str)
all_sales['month'] = all_sales['month'].astype(str)
all_sales['day'] = all_sales['day'].astype(str)
all_sales['day_of_week'] = all_sales['day_of_week'].astype(str)

: 

In [10]:
all_sales = all_sales.fillna('unknown')

In [11]:
all_sales.isna().sum()

date           0
store_id       0
item_id        0
quantity       0
price_base     0
year           0
month          0
day            0
day_of_week    0
dept_name      0
class_name     0
format         0
city           0
dtype: int64

In [12]:
def train_val_test_split(df, target, train_size, val_size, test_size, random_state):
    
    df_full_train, df_test = train_test_split(df, test_size=test_size, random_state=random_state)
    val_portion = val_size / (train_size + val_size)
    df_train, df_val = train_test_split(df_full_train, test_size=val_portion, random_state=random_state)

    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)

    y_train = df_train[target].values
    y_val = df_val[target].values
    y_test = df_test[target].values

    del df_train[target]
    del df_val[target]
    del df_test[target]

    return df_full_train, df_train, df_test, df_val, y_train, y_val, y_test

In [13]:
df_full_train, df_train, df_test, df_val, y_train, y_val, y_test = \
    train_val_test_split(df=all_sales, target='quantity', train_size=0.6, val_size=0.2, test_size=0.2, random_state=1)

In [14]:
def train(df_train, y_train, max_depth):
    dicts = df_train.to_dict(orient='records')

    dv = DictVectorizer(sparse=True)
    X_train = dv.fit_transform(dicts)

    model = DecisionTreeRegressor(max_depth=max_depth)
    model.fit(X_train, y_train)
    
    return dv, model

In [15]:
dv, model = train(df_train=df_train, y_train=y_train, max_depth=1)
# importances = list(zip(dv.feature_names_, model.feature_importances_))
# df_importance = pd.DataFrame(importances, columns=['feature', 'gain'])
# df_importance = df_importance.sort_values(by='gain', ascending=False)
# df_importance

In [16]:
def train_rf(df_train, y_train, random_state, n_estimators):
    dicts = df_train.to_dict(orient='records')

    dv = DictVectorizer(sparse=True)
    X_train = dv.fit_transform(dicts)

    model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state, n_jobs=-1)
    model.fit(X_train, y_train)
    
    return dv, model

def predict(df, dv, model):
    dicts = df.to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict(X)

    return y_pred

def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [17]:
y_pred = predict(df=df_val, dv=dv, model=model)
round(rmse(y_val, y_pred),3)

np.float64(21.605)

In [18]:
# dv, rf_model = train_rf(df_train=df_train, y_train=y_train, random_state=1, n_estimators=)
# y_pred = predict(df=df_val, dv=dv, model=rf_model)
# round(rmse(y_val, y_pred),3)

In [20]:
def prepare_test_data(test_df, catalog_df, stores_df):
    # Convert date to datetime
    test_df['date'] = pd.to_datetime(test_df['date'], format='%d.%m.%Y')
    
    # Create time features
    test_df['year'] = test_df['date'].dt.year
    test_df['month'] = test_df['date'].dt.month
    test_df['day'] = test_df['date'].dt.day
    test_df['day_of_week'] = test_df['date'].dt.dayofweek
    
    # Merge with catalog and stores
    test_df = test_df.merge(catalog[['item_id', 'dept_name', 'class_name']], 
                           on='item_id', how='left')
    test_df = test_df.merge(stores[['store_id', 'format', 'city']], 
                           on='store_id', how='left')
    
    # Convert to string type
    test_df['date'] = test_df['date'].astype(str)
    test_df['store_id'] = test_df['store_id'].astype(str)
    test_df['year'] = test_df['year'].astype(str)
    test_df['month'] = test_df['month'].astype(str)
    test_df['day'] = test_df['day'].astype(str)
    test_df['day_of_week'] = test_df['day_of_week'].astype(str)
    
    # Fill missing values
    test_df = test_df.fillna('unknown')
    
    return test_df

# Process test data and make predictions
processed_test = prepare_test_data(test, catalog, stores)

In [21]:
processed_test.head()

,row_id,item_id,store_id,date,year,month,day,day_of_week,dept_name,class_name,format,city
0,0,c578da8e8841,1,2024-09-27,2024,9,27,4,ТАБАЧНЫЕ ИЗДЕЛИЯ,СИГАРЕТЫ,Format-1,City1
1,1,c578da8e8841,1,2024-09-28,2024,9,28,5,ТАБАЧНЫЕ ИЗДЕЛИЯ,СИГАРЕТЫ,Format-1,City1
2,2,c578da8e8841,1,2024-09-29,2024,9,29,6,ТАБАЧНЫЕ ИЗДЕЛИЯ,СИГАРЕТЫ,Format-1,City1
3,3,c578da8e8841,1,2024-09-30,2024,9,30,0,ТАБАЧНЫЕ ИЗДЕЛИЯ,СИГАРЕТЫ,Format-1,City1
4,4,c578da8e8841,1,2024-10-01,2024,10,1,1,ТАБАЧНЫЕ ИЗДЕЛИЯ,СИГАРЕТЫ,Format-1,City1


In [22]:

test_predictions = predict(processed_test, dv, model)

# Create submission dataframe
submission = pd.DataFrame({
    'row_id': test['row_id'],
    'quantity': test_predictions
})

# Optional: save predictions
# submission.to_csv('predictions.csv', index=False)

In [25]:
# Optional: save predictions
submission.to_csv('predictions_decision_tree.csv', index=False)